# Carga de librerías

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('ggplot')
import os
import zipfile
import requests
from io import BytesIO
# print(plt.style.available)
from scipy.spatial import distance_matrix
from datetime import datetime, timedelta
import re
import numpy as np

# Obtención de los conjuntos de datos

## Accidentes con bicicletas implicadas (2019-2023)

En primer lugar, se va a cargar el conjunto de datos de accidentes con bicicletas implicadas disponible en el portal de datos abiertos del Ayuntamiento de Madrid. Conviene destacar que los datos están disponibles tanto en formato .csv como .xlsx. La periodicidad de cada fichero es anual, si bien es cierto que hay un decalaje de algo más de un mes por lo que los datos más recientes son los del 31/05/2023.

Respecto a la estructura de los datos, existe un registro por persona implicada en el accidente (es decir, que si en un accidente se han visto implicadas dos personas, habrá dos registros). Existe histórico desde 2010 hasta la actualidad, aunque en este caso se ha optado por capturar solo el histórico desde 2019 hasta la fecha, ya que en 2019 la estructura de los datos cambió. De esta forma, según la documentación disponible en la página web del portal de datos abiertos (https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=20f4a87ebb65b510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default), la estructura de los datos es la siguiente:
* **num_expediente**: Número de experiente del accidente. Sigue el patrón AAAASNNNNNN, donde: AAAA es el año del accidente, S cuando se trata de un expediente con accidente y NNNNNN es un número correlativo por año. Como ya se comentó, puede haber más de un registro con el mismo número de expediente si ha habido varios afectados en un mismo accidente.
* **fecha**: Fecha, en formato dd/mm/aaaa.
* **hora**: La hora se establece en rangos horarios de 1 hora.
* **localizacion**: Recoge el lugar del accidente. Suele seguir la estructura calle 1 ‐ calle 2 (para cruces) o una calle.
* **numero**: Número de la calle, cuando la vía tiene sentido.
* **cod_distrito**: Código único que sirve para identificar el distrito donde tuvo lugar el accidente.
* **distrito**: Nombre del distrito.
* **tipo_accidente**: Puede tomar los siguientes valores:
    * **Colisión doble**: Accidente de tráfico ocurrido entre dos vehículos en movimiento, (colisión frontal, fronto lateral, lateral).
    * **Colisión múltiple:**: Accidente de tráfico ocurrido entre más de dos vehículos en movimiento.
    * **Alcance**: Accidente que se produce cuando un vehículo circulando o detenido por las circunstancias del tráfico es golpeado en su parte posterior por otro vehículo.
    * **Choque contra obstáculo o elemento de la vía**: Accidente ocurrido entre un vehículo en movimiento con conductor y un objeto inmóvil que ocupa la vía o zona apartada de la misma, ya sea vehículo estacionado, árbol, farola, etc.
    * **Atropello a persona**: Accidente ocurrido ente un vehículo y un peatón que ocupa la calzada o que transita por aceras, refugios, paseos o zonas de la vía pública no destinada a la circulación de vehículos.
    * **Vuelco**: Accidente sufrido por un vehículo con más de dos ruedas y que por alguna circunstancia sus neumáticos pierden el contacto con la calzada quedando apoyado sobre un costado o sobre el techo.
    * **Caída**: Se agrupan todas las caídas relacionadas con el desarrollo y las circunstancias del tráfico, (motocicleta, ciclomotor, bicicleta, viajero bus, etc.).
    * **Otras causas**: Recoge los accidentes por atropello a animal, despeñamiento, salida de la vía, y otros.
* **estado_meteorologico**: Condiciones ambientales presentes en el momento del accidente (nublado, despejado...).
* **tipo_vehiculo**: Tipo de vehículo implicado (bicicleta, coche...).
* **tipo_persona**: Rol de la persona implicada, a saber:: Conductor, peatón, testigo o viajero.
* **rango_edad**: Tramo de edad de la persona implicada en el accidente.
* **sexo**: Puede tomar los siguiente valores: Hombre, mujer o no asignado.
* **cod_lesividad**: Código de la lesividad, tipoficado a continuación:
    * 01: Atención en urgencias sin posterior ingreso. - LEVE
    * 02: Ingreso inferior o igual a 24 horas - LEVE
    * 03: Ingreso superior a 24 horas. - GRAVE
    * 04: Fallecido 24 horas - FALLECIDO
    * 05: Asistencia sanitaria ambulatoria con posterioridad - LEVE
    * 06: Asistencia sanitaria inmediata en centro de salud o mutua - LEVE
    * 07: Asistencia sanitaria sólo en el lugar del accidente - LEVE
    * 14: Sin asistencia sanitaria
    * 77: Se desconoce
    * (En blanco): Sin asistencia sanitaria
* **lesividad**: Descripción de la lesividad. Ver el campo anterior.
* **coordenada_x_utm**: Coordenada X del lugar del accidente en el sistema de referencia de coordenadas ETRS89 / UTM zone 30N (EPSG: 25830).
* **coordenada_y_utm**: Coordenada Y del lugar del accidente en el sistema de referencia de coordenadas ETRS89 / UTM zone 30N (EPSG: 25830).
* **positiva_alcohol**: Indica si la persona involucrada dio positivo (S) en la prueba de alcohol o negativo (N).
* **positiva_droga**: Indica si la persona involucrada dio positivo (1.0) en la prueba de drogas o negativo (en blanco).




In [2]:
# Enlace para descargar cada dataset de accidentes con bicicletas implicadas
accidentes_url = 'https://datos.madrid.es/egob/catalogo/300110-{id}-accidentes-bicicleta.csv'

# Definimos una función para quitar las tíldes. Nos servirá para quitar las tíldes de los nombres de las columnas
def quitar_tildes(texto):
    tildes = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'
    }
    texto_limpio = ''.join(tildes.get(c, c) for c in texto)
    return texto_limpio

# Empezamos cargando los datos de 2023 a partir del enlace web
accidentes_raw_df = pd.read_csv(accidentes_url.format(id=26), delimiter=';', decimal=',')
# Quitamos las tildes de los nombres de las columnas
columnas_sin_tilde = [quitar_tildes(columna) for columna in accidentes_raw_df.columns]
accidentes_raw_df.columns = columnas_sin_tilde

# Ahora cargamos los demás datasets (de 2019 a 2022) y lo concatenamos al dataset inicial
for id in range(18,26,2):
    df_temp = pd.read_csv(accidentes_url.format(id=id), delimiter=';', decimal=',')
    # Dado que hay un error en los datasets de 2019 a 2022 en las columnas de coordenadas, es necesario convertirlas de texto a número quitando los puntos y dividiendo entre 1000
    df_temp['coordenada_x_utm'] = df_temp['coordenada_x_utm'].str.replace('.', '').astype(float)/1000
    df_temp['coordenada_y_utm'] = df_temp['coordenada_y_utm'].str.replace('.', '').astype(float)/1000
    # Se va a quitar las tíldes de los nombres de las columnas
    columnas_sin_tilde = [quitar_tildes(columna) for columna in df_temp.columns]
    df_temp.columns = columnas_sin_tilde
    # Se unifican las columnas lesividad y tipo_lesividad en lesividad
    if 'tipo_lesividad' in df_temp.columns:
        df_temp['lesividad'] = df_temp['tipo_lesividad']
        df_temp.drop(['tipo_lesividad'], axis=1, inplace=True)
    # Concatenamos df_temp al dataset inicial
    accidentes_raw_df = pd.concat([accidentes_raw_df, df_temp], ignore_index=True)

# Se añaden una serie de campos que serán necesarios para futuros pasos
# Se añade la columna que concatene el Año y el Mes
accidentes_raw_df['AnoMes'] = pd.to_datetime(accidentes_raw_df['fecha'], format='%d/%m/%Y').dt.strftime('%Y-%m')

# Se añade la columna que concatene la fecha y la hora, quitando los segundos y redondeando al cuarto de hora más próximo por defecto
# El formato de esta columna será 2019-01-01 00:00:00
accidentes_raw_df['fecha_hora'] = pd.to_datetime(accidentes_raw_df['fecha'] + ' ' + accidentes_raw_df['hora'], format='%d/%m/%Y %H:%M:%S').dt.floor('15min').dt.strftime('%Y-%m-%d %H:%M:00')

# Se da el formato correcto a las columnas de coordenadas
accidentes_raw_df['coordenada_x_utm'] = accidentes_raw_df['coordenada_x_utm'].astype(float)
accidentes_raw_df['coordenada_y_utm'] = accidentes_raw_df['coordenada_y_utm'].astype(float)

# Vamos a guardar el dataset en fichero CSV para tenerlo en local
accidentes_raw_df.to_csv('./Datasets/accidentes_2019_2023.csv', sep=';', encoding='latin-1', index=False)

C:\Users\alke-\AppData\Local\Temp\ipykernel_8028\673297701.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_temp['coordenada_x_utm'] = df_temp['coordenada_x_utm'].str.replace('.', '').astype(float)/1000
C:\Users\alke-\AppData\Local\Temp\ipykernel_8028\673297701.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_temp['coordenada_y_utm'] = df_temp['coordenada_y_utm'].str.replace('.', '').astype(float)/1000
C:\Users\alke-\AppData\Local\Temp\ipykernel_8028\673297701.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=Tr

In [3]:
# Se realizan varias comprobaciones sobre el dataset, como ver su información básica
accidentes_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   num_expediente        4018 non-null   object 
 1   fecha                 4018 non-null   object 
 2   hora                  4018 non-null   object 
 3   localizacion          4018 non-null   object 
 4   numero                4018 non-null   object 
 5   cod_distrito          4018 non-null   int64  
 6   distrito              4018 non-null   object 
 7   tipo_accidente        4018 non-null   object 
 8   estado_meteorologico  3892 non-null   object 
 9   tipo_vehiculo         4018 non-null   object 
 10  tipo_persona          4018 non-null   object 
 11  rango_edad            4018 non-null   object 
 12  sexo                  4018 non-null   object 
 13  cod_lesividad         3484 non-null   float64
 14  lesividad             3484 non-null   object 
 15  coordenada_x_utm     

In [4]:
# Comprobamos de forma más sencilla donde hay valores nulos
accidentes_raw_df.isnull().sum()

num_expediente             0
fecha                      0
hora                       0
localizacion               0
numero                     0
cod_distrito               0
distrito                   0
tipo_accidente             0
estado_meteorologico     126
tipo_vehiculo              0
tipo_persona               0
rango_edad                 0
sexo                       0
cod_lesividad            534
lesividad                534
coordenada_x_utm           0
coordenada_y_utm           0
positiva_alcohol           7
positiva_droga          4010
AnoMes                     0
fecha_hora                 0
dtype: int64

In [5]:
# Comprobamos rápidamente que los campos numéricos tienen los valores esperados
accidentes_raw_df.describe()

,cod_distrito,cod_lesividad,coordenada_x_utm,coordenada_y_utm,positiva_droga
count,4018.000000,3484.000000,4018.000000,4.018000e+03,8.0
mean,8.331010,5.837830,441649.178399,4.475131e+06,1.0
std,5.800212,3.898194,3354.793170,3.750375e+03,0.0
min,1.000000,1.000000,430367.536000,4.464458e+06,1.0
25%,3.000000,2.000000,439681.716750,4.472901e+06,1.0
50%,8.000000,6.000000,441140.659500,4.474747e+06,1.0
75%,13.000000,7.000000,443217.807250,4.476912e+06,1.0
max,21.000000,14.000000,454614.270000,4.490695e+06,1.0


In [89]:
# Se observa que las columnas de coordenadas toman valores coherentes para ser coordenadas utm
# Esto es clave ya que luego se van a utilizar estos campos

In [6]:
accidentes_raw_df.tipo_persona.value_counts()

Conductor    3734
Peatón        263
Pasajero       21
Name: tipo_persona, dtype: int64

#### Se añade el número de implicados
Esto viene de 1. Procesamiento v2 desde el inicio hasta la celda que pone HASTA AQUÍ LO QUE SE PUEDE APROVECHAR PARA 1. Procesamiento.ipynb

In [7]:
implicados = pd.read_csv('./Datasets/accidentes_con_implicados.csv', delimiter=';', encoding='latin-1')

In [8]:
implicados.tipo_accidente.value_counts()

Caída                           1327
Colisión fronto-lateral          959
Alcance                          549
Atropello a persona              482
Colisión lateral                 420
Colisión frontal                 189
Choque contra obstáculo fijo     119
Otro                              34
Atropello a animal                25
Colisión múltiple                 18
Vuelco                             5
Solo salida de la vía              3
Name: tipo_accidente, dtype: int64

In [9]:
implicados = implicados.drop_duplicates(subset='num_expediente')

In [10]:
implicados.columns

Index(['index', 'num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorologico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga', 'AnoMes', 'fecha_hora', 'tipo_vehiculo_agg',
       'implicados', 'implicados_peatones', 'implicados_pesados',
       'implicados_turismos', 'implicados_motocicletas',
       'implicados_bicicletas', 'implicados_otros', 'implicados_epac',
       'implicados_ligeros'],
      dtype='object')

In [11]:
implicados = implicados[['num_expediente', 'implicados', 'implicados_peatones', 'implicados_pesados', 'implicados_turismos',
       'implicados_motocicletas', 'implicados_bicicletas', 'implicados_otros',
       'implicados_epac', 'implicados_ligeros']]

In [12]:
accidentes_raw_df = pd.merge(accidentes_raw_df,implicados, on=['num_expediente'], how='left')

In [13]:
accidentes_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4018 entries, 0 to 4017
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num_expediente           4018 non-null   object 
 1   fecha                    4018 non-null   object 
 2   hora                     4018 non-null   object 
 3   localizacion             4018 non-null   object 
 4   numero                   4018 non-null   object 
 5   cod_distrito             4018 non-null   int64  
 6   distrito                 4018 non-null   object 
 7   tipo_accidente           4018 non-null   object 
 8   estado_meteorologico     3892 non-null   object 
 9   tipo_vehiculo            4018 non-null   object 
 10  tipo_persona             4018 non-null   object 
 11  rango_edad               4018 non-null   object 
 12  sexo                     4018 non-null   object 
 13  cod_lesividad            3484 non-null   float64
 14  lesividad               

In [14]:
accidentes_raw_df[accidentes_raw_df['implicados'].isna()].tipo_persona.value_counts()

Series([], Name: tipo_persona, dtype: int64)

#### Mejora del campo localización

In [99]:
# A continuación, se va a proceder a mejorar el campo localizacion, de cara a obtener información útil al respecto
# Lo primero que se va a hacer es eliminar las tildes del campo y convertir todo a mayúsculas
accidentes_raw_df["localizacion"] = accidentes_raw_df["localizacion"].apply(quitar_tildes).str.upper()

In [100]:
# Seguidamente, se van a quitar los espacios antes y después del texto
accidentes_raw_df["localizacion"] = accidentes_raw_df["localizacion"].str.strip()

In [101]:
# Tras revisar los registros, se observa que se pueden eliminar los puntos ya que no aportan nada
accidentes_raw_df["localizacion"] = accidentes_raw_df["localizacion"].apply(lambda x: re.sub(r'\.', '', x))

In [102]:
# También se van a quitar los espacios en blanco de más
accidentes_raw_df["localizacion"] = accidentes_raw_df["localizacion"].apply(lambda x: re.sub(r'\s+', ' ', x))

In [103]:
# Se va a crear un filtro para ver qué registros contienen " - ", " / ", " -- ", " CON "
filtro = accidentes_raw_df["localizacion"].str.contains(r' - | -- | / |\sCON\s', case=False, regex=True)

# Se muestra qué porcentaje de registros cumplen el patrón y, por tanto, son accidentes en cruces
print(f"Un {(accidentes_raw_df[filtro].count()[0]/ accidentes_raw_df.count()[0])*100:.2f} % de los registros tienen como localización un cruce")

Un 43.55 % de los registros tienen como localización un cruce


In [104]:
# Para simplificar el tratamiento, se va a separar la columna localización es dos columnas a partir de los patrones " / ", " - ", " -- " o " CON "
accidentes_raw_df[['localizacion_1', 'localizacion_2']] = accidentes_raw_df["localizacion"].str.split(r' - | -- | / |\sCON\s', expand=True, n=1)
# Se rellenan los vacíos de la segunda columna con (vacío) para facilitar las funciones con texto
accidentes_raw_df['localizacion_2'].fillna('',inplace=True)

In [105]:
# Se vuelven a quitar los espacios de más y los espacios antes y después del texto, en este caso para las dos columnas nuevas
accidentes_raw_df["localizacion_1"] = accidentes_raw_df["localizacion_1"].str.strip().apply(lambda x: re.sub(r'\s+', ' ', x))
accidentes_raw_df["localizacion_2"] = accidentes_raw_df["localizacion_2"].str.strip().apply(lambda x: re.sub(r'\s+', ' ', x))

In [106]:
# A continuación, se quiere obtener para cada localización el tipo de vía, el nombre de la vía y el número
# Para ello, se va a definir una función que permita limpiar la dirección y otra que analice la dirección resultante para obtener el tipo de vía, su nombre y el número
                       
def limpiar_direccion(direccion):
    """
    Función que limpia la dirección, quitando una serie de stopwords, los textos entre paréntesis, el símbolo + y algún otro patrón como que empiece por la palabra FRENTE
    """
    # Se eliminan las stopwords y palabras específicas
    stopwords = ["S/N", "NUM", "KM", "PK", "Nº"]
    direccion = re.sub(r'\b(?:' + '|'.join(map(re.escape, stopwords)) + r')\b', '', direccion, flags=re.IGNORECASE)
    direccion = re.sub(r'\bCRUCE\b', '', direccion, flags=re.IGNORECASE)

    # Se elimina el texto entre paréntesis
    direccion = re.sub(r'\(.*?\)', '', direccion)

    # Se elimina el símbolo +
    direccion = re.sub(r'\+', '', direccion)

    # Se separa la dirección por palabras
    palabras = direccion.split()

    # Se busca la posición de "FRENTE"
    indice_frente = palabras.index("FRENTE") if "FRENTE" in palabras else -1

    # Se elimina todo lo que aparece después de "FRENTE" solo si "FRENTE" no es la primera palabra, ya que si no, eliminamos la dirección completa
    if indice_frente > 0:
        direccion = ' '.join(palabras[:indice_frente])

    # Se elimina todo lo que aparece después de "FRENTE"
    #direccion = re.split(r'\bFRENTE\b', direccion, flags=re.IGNORECASE)[0]

    return direccion.strip()

def analizar_direccion(direccion):
    """
    Función que a partir de una dirección, la analiza para obtener el tipo de vía, el nombre y el número de la misma
    """
    # Se convierte todo a mayúscula
    direccion = direccion.upper()

    # Condición para evitar errores de direcciones vacías
    if len(direccion) < 2:
        return None, None, None

    # Se definen las variables
    tipo_via = None
    patron_via = None

    # Se definen los patrones posibles
    patrones = {
        "CALLE": ["CALLE", "CALL", "CL", "C/"],
        "AVENIDA": ["AVENIDA", "AVDA", "AVD", "AV"],
        "PLAZA": ["PLAZA"],
        "ARROYO": ["ARROYO", "ARRY"],
        "AUTOPISTA": ["AUTOPISTA", "AUTOP"],
        "AUTOVÍA": ["AUTOVÍA", "AUTOV"],
        "BULEVAR": ["BULEV"],
        "CARRETERA": ["CARRETERA", "CARETAR", "CRTA", "CTRA", "CRA"],
        "ACCESO": ["ACCESO"],
        "PASEO": ["PASEO"],
        "GLORIETA": ["GLORIETA", "GTA"],
        "PLAZA": ["PLAZA", "PLZA", "PZA"],
        "RONDA": ["RONDA", "ROND", "RDA"],
        "COSTANILLA": ["COSTANILLA", "CSTAN"],
        "CAMINO": ["CAMINO", "CMNO"],
        "CAÑADA": ["CAÑADA", "CÑADA", "CNADA"],
        "CUESTA": ["CUESTA", "CUSTA"],
        "EDIFICIO": ["EDIFICIO", "EDIFI"],
        "PUENTE": ["PUENTE", "PNTE", "PTE"],
        "LUGAR": ["LUGAR"],
        "PISTA": ["PISTA"],
        "POLÍGONO": ["POLIGONO", "POLIG"],
        "PARQUE": ["PARQUE", "PQUE"],
        "PUERTA": ["PUERTA", "PTA"],
        "TRAVESÍA": ["TRAVESIA", "TRVA"]
    }
    
    # Se limpia la dirección
    direccion_limpia = limpiar_direccion(direccion)

    # Se buscan los patrones en la direccion
    for t_via, lista_patrones in patrones.items():
        for patron in lista_patrones:
            if re.match(f"^{patron}$", direccion_limpia.split()[0], re.IGNORECASE):
                tipo_via = t_via
                patron_via = patron
                break
    
    # Se definen más variables
    nombre_via = direccion_limpia
    numero_via = None

    # Se elimina el patrón de tipo de vía al inicio
    if patron_via:
        nombre_via = re.sub(rf"^{patron_via}\s*", '', nombre_via, flags=re.IGNORECASE).strip()

    # Se elimina "DE" y "DEL" del inicio del texto
    nombre_via = re.sub(r'^\bDE\b|\bDEL\b', '', nombre_via, flags=re.IGNORECASE).strip()

    if tipo_via in ["CARRETERA", "AUTOPISTA", "AUTOVÍA"]:
        # Se busca el patrón "M-", "A-", "N-" o "R-" seguido de uno o varios números
        nombre_match = re.search(r'^(M-|A-|N-|R-)\d+', direccion_limpia.split(',')[0])
        if nombre_match:
            nombre_via = nombre_match.group()
            direccion_sin_nombre = re.sub(rf"{nombre_via}", '', direccion_limpia).strip()
            numero_match = re.search(r'(?:\+)?\d+', direccion_sin_nombre)
            if numero_match:
                numero_via = numero_match.group()
    else:
        # Se busca el número de la vía
        numero_match = re.search(r'(?:\+)?\d+', direccion_limpia)
        if numero_match:
            numero_via = numero_match.group()
            # nombre_via = direccion_limpia.replace(numero_via, '').strip()
            nombre_via = re.sub(rf"{numero_via}.*", '', nombre_via, flags=re.IGNORECASE).strip()

    # Se eliminan las comas al final del texto
    nombre_via = re.sub(r',\s*', '', nombre_via.strip())

    # Se eliminan los espacios en blanco múltiples
    nombre_via = re.sub(r'\s+', ' ', nombre_via)

    # Se eliminan los espacios al inicio y al final
    nombre_via = nombre_via.strip()

    # Se vuelve a limpiar el nombre_via
    nombre_via = limpiar_direccion(nombre_via)
    
    return tipo_via, nombre_via, numero_via

In [107]:
# Seguidamente, se aplica a las columnas localizacion_1 y localizacion_2
accidentes_raw_df[['clase_via_1','nombre_via_1', 'numero_via_1']] = accidentes_raw_df["localizacion_1"] .apply(lambda x: pd.Series(analizar_direccion(x)))
accidentes_raw_df[['clase_via_2','nombre_via_2', 'numero_via_2']] = accidentes_raw_df["localizacion_2"] .apply(lambda x: pd.Series(analizar_direccion(x)))

In [108]:
# Vamos a guardar el dataset en fichero CSV para tenerlo en local
accidentes_raw_df.to_csv('./Datasets/accidentes_con_vias_2019_2023.csv', sep=';', encoding='latin-1', index=False)

## Ubicación de los puntos de medida del tráfico (2019-2023)

Una vez cargados los accidentes con implicación de bicicletas desde el 2019 hasta la actualidad, se echa de menos algún atributo que indique la intensidad del tráfico para cada accidente. Por ello, se opta por enriquecer el conjunto de datos anterior con el histórico de tráfico que tiene disponible el Portal abierto del Ayuntamiento de Madrid. Como paso previo a añadir una columna que especifique la intensidad de tráfico en el lugar y momento del accidente, es necesario identificar el punto de medida del tráfico más próximo a cada lugar del accidente en el instante en el que tuvo lugar.

De esta forma, se carga el conjunto de datos de los puntos de medida del tráfico accesible desde la página web del Portal abierto del Ayuntamiento de Madrid. De forma mensual, en esta página web se cuelga la localización y la información básica de los distintos puntos de medida existentes en la ciudad de Madrid. La información está disponible tanto en formato shapefile (SHP) como csv, y su contenido es el siguiente:
* **tipo_elem**: Permite distinguir entre dispositivos de control semafórico (URB) y dispositivos de vías rápidas y accesos a Madrid (M-30).
* **distrito**: Código único que sirve para identificar el distrito donde se ubica el punto de medida.
* **id**: Identificador secuencial, único e invariable del punto de medida. Este es el campo que permitirá relacionar cada accidente con la intensidad de tráfico.
* **cod_cent**: Código centralizado del punto de medida.
* **nombre**: La denominación de los puntos de medida sigue una nomenclatura común:
    * Para los puntos de medida del tráfico en áreas urbanas, se utiliza la calle y la dirección del flujo de circulación.
    * Para los puntos de medición del tráfico en vías rápidas y accesos a Madrid, se emplea el punto kilométrico, la calzada y se indica si es la vía central, la vía de servicio o un enlace.
* **utm_x**: Coordenada X del punto que indica la representación del punto de medida en el sistema de referencia de coordenadas ETRS89 / UTM zone 30N (EPSG: 25830).
* **utm_y**: Coordenada Y del punto que indica la representación del punto de medida en el sistema de referencia de coordenadas ETRS89 / UTM zone 30N (EPSG: 25830).
* **longitud**: Longitud en el sistema de referencia de coordenadas WGS 84 (EPSG:4326).
* **latitud**: Latitud en el sistema de referencia de coordenadas WGS 84 (EPSG:4326).

In [109]:
# Enlace para descargar cada dataset con la ubicación de los puntos de medida del tráfico
pmed_url = 'https://datos.madrid.es/egob/catalogo/202468-{id}-intensidad-trafico.csv'

# Empezamos cargando los datos de enero de 2019 (2019-01), ya que los conjuntos de datos están disponibles con periodicidad mensual
pmed_raw_df = pd.read_csv(pmed_url.format(id=31), delimiter=';', encoding='utf-8', decimal='.')

# Añadimos la columna que nos ayudará a identificar el año-mes de los puntos de medida del tráfico
pmed_raw_df['AnoMes'] = '2019-01'

In [110]:
# A diferencia del conjunto de datos de accidentes con bicicletas implicadas, el id de la url no sigue un orden coherente, por lo que
# se va a definir un diccionario que relacione cada id con el año-mes
diccionario_pmed = {
    31:'2019-01',
    34:'2019-02',
    37:'2019-03',
    40:'2019-04',
    43:'2019-05',
    46:'2019-06',
    49:'2019-07',
    52:'2019-08',
    55:'2019-09',
    58:'2019-10',
    61:'2019-11',
    64:'2019-12',
    67:'2020-01',
    71:'2020-02',
    68:'2020-03',
    74:'2020-04',
    77:'2020-05',
    80:'2020-06',
    83:'2020-07',
    86:'2020-08',
    89:'2020-09',
    92:'2020-10',
    95:'2020-11',
    98:'2020-12',
    101:'2021-01',
    104:'2021-02',
    107:'2021-03',
    110:'2021-04',
    113:'2021-05',
    116:'2021-06',
    119:'2021-07',
    122:'2021-08',
    126:'2021-09',
    128:'2021-10',
    131:'2021-11',
    134:'2021-12',
    137:'2022-01',
    140:'2022-02',
    143:'2022-03',
    146:'2022-04',
    149:'2022-05',
    152:'2022-06',
    155:'2022-07',
    158:'2022-08',
    161:'2022-09',
    164:'2022-10',
    167:'2022-11',
    170:'2022-12',
    173:'2023-01',
    176:'2023-02',
    179:'2023-03',
    182:'2023-04',
    185:'2023-05',
    188:'2023-06'
}

In [111]:
# Probramos con diferentes codificaciones hasta encontrar la correcta para cada dataset, ya que se ha observado que
# la codificación cambia para cada mes
encodings = ['utf-8', 'latin-1', 'ISO-8859-1']

# Ahora cargamos los demás datasets (de 2019-02 a 2023-06) y lo concatenamos al dataset inicial
for x in diccionario_pmed:
    for encoding in encodings:
        try:
            df_temp = pd.read_csv(pmed_url.format(id=x), delimiter=';', encoding=encoding, decimal='.')
            break
        except UnicodeDecodeError:
            continue
    df_temp['AnoMes'] = diccionario_pmed[x]
    pmed_raw_df = pd.concat([pmed_raw_df, df_temp], ignore_index=True)
    # Mostramos en pantalla el año-mes que se está cargando, y el encoding aplicado
    print(diccionario_pmed[x], encoding)

# Vamos a guardar el dataset en fichero CSV para tenerlo en local
pmed_raw_df.to_csv('./Datasets/pmed_ubicacion_2019_2023.csv', sep=';', encoding='utf-8', index=False)

2019-01 utf-8
2019-02 utf-8
2019-03 utf-8
2019-04 utf-8
2019-05 latin-1
2019-06 latin-1
2019-07 utf-8
2019-08 utf-8
2019-09 utf-8
2019-10 utf-8
2019-11 utf-8
2019-12 utf-8
2020-01 utf-8
2020-02 utf-8
2020-03 utf-8
2020-04 utf-8
2020-05 utf-8
2020-06 utf-8
2020-07 utf-8
2020-08 utf-8
2020-09 utf-8
2020-10 utf-8
2020-11 utf-8
2020-12 utf-8
2021-01 utf-8
2021-02 utf-8
2021-03 utf-8
2021-04 utf-8
2021-05 utf-8
2021-06 utf-8
2021-07 utf-8
2021-08 utf-8
2021-09 utf-8
2021-10 utf-8
2021-11 utf-8
2021-12 utf-8
2022-01 utf-8
2022-02 utf-8
2022-03 utf-8
2022-04 utf-8
2022-05 utf-8
2022-06 utf-8
2022-07 utf-8
2022-08 utf-8
2022-09 utf-8
2022-10 utf-8
2022-11 utf-8
2022-12 utf-8
2023-01 utf-8
2023-02 utf-8
2023-03 utf-8
2023-04 utf-8
2023-05 utf-8
2023-06 utf-8


In [112]:
# Al igual que se hizo en el dataset anterior, comprobamos rápidamente que los datos numéricos tienen los valores esperados
pmed_raw_df.describe()

,distrito,id,utm_x,utm_y,longitud,latitud
count,243912.000000,244149.000000,244149.000000,2.441490e+05,244149.000000,244149.000000
mean,9.822018,6209.274148,441714.313788,4.475644e+06,-3.687142,40.429381
std,5.302096,2295.241488,3474.518011,4.190451e+03,0.040898,0.037789
min,1.000000,0.000000,429055.947346,4.464902e+06,-3.836943,40.332454
25%,5.000000,4534.000000,439491.253385,4.472310e+06,-3.713128,40.399064
50%,9.000000,5692.000000,441469.594638,4.475791e+06,-3.690123,40.430855
75%,15.000000,6839.000000,443993.831389,4.478871e+06,-3.660338,40.458456
max,21.000000,11127.000000,450772.278972,4.485213e+06,-3.580713,40.515611


In [113]:
# Se analiza el tipo de cada columna, así como los nulos de cada columna
pmed_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293608 entries, 0 to 293607
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   tipo_elem  244149 non-null  object 
 1   distrito   243912 non-null  float64
 2   id         244149 non-null  float64
 3   cod_cent   244149 non-null  object 
 4   nombre     243461 non-null  object 
 5   utm_x      244149 non-null  float64
 6   utm_y      244149 non-null  float64
 7   longitud   244149 non-null  float64
 8   latitud    244149 non-null  float64
 9   AnoMes     293608 non-null  object 
dtypes: float64(6), object(4)
memory usage: 22.4+ MB


In [114]:
# Se confirma que las columnas de coordenadas tiene el formato correcto, por lo que ya se puede avanzar

## Histórico de datos del tráfico desde 2019

Hasta ahora, si despone de un conjunto de datos con todos los accidentes con implicación de bicicletas, y otro, con la ubicación de los puntos de medida por mes, para el periodo completo de análisis (de 2019 hasta la actualidad). Llegados a este punto, solo queda añadir a cada accidente el histórico de datos de tráfico teniendo en cuenta la fecha y hora del accidente, así como el id del punto de medida más próximo. Al igual que hasta hora, desde el portal de datos abiertos del Ayuntamiento de Madrid se puede descargar esta información. 

De esta forma, el conjunto de datos que hay que cargar es el de datos históricos de los puntos de medida de tráfico. En este caso, el decalaje es de un mes, y los datos se capturan en los distintos puntos de medida y se integran en la base de datos SICTRAM cada 15 minutos. Los datos están disponibles en ficheros csv mensuales, cuya estructura es la siguiente:

* **id**: Identificador secuencial, único e invariable del punto de medida. Este es el campo que permitirá relacionar cada registro con el punto de medida de tráfico correspondiente.
* **fecha**: Fecha y hora oficiales de Madrid con formato dd/mm/yyyy hh:mi:ss
* **tipo_elem**: Permite distinguir entre dispositivos de control semafórico o urbanos (URB) y dispositivos de vías rápidas y accesos a Madrid (M-30).
* **intensidad**: Número de vehículos que discurren por un punto de medida en cada hora. Dado que sus unidades son vehículos/hora, para obtener el valor efectivo de vehículos que han circulado en ese intervalo de 15 minutos, hay que dividir entre cuatro el número disponible. Un valor negativo se interpreta como la ausencia de datos.
* **ocupacion**: Porcentaje de tiempo que está un punto de medida de tráfico ocupado por un vehículo. De esta forma, si un vehículo ha estado situado durante 7,5 minutos frente a un punto de medida, la ocupación registrada en un periodo de 15 minutos será del 50%. Un valor negativo se interpreta como la ausencia de datos.
* **carga**: Parámetro de carga de la vía en el intervalo de 15 minutos. Se trata de una medida sintética utilizada para estimar el nivel de congestión, calculado a partir de un algoritmo que usa como variables la intensidad y la ocupación, con ciertos factores de corrección. Establece el grado de uso de la vía en un rango de 0 (vacía) a 100 (colapso). Un valor negativo significa la ausencia de datos. En puntos de medida de tipo M-30 no se utiliza el parámetro de carga, ya que no hay regulación semafórica, por lo que su valor será nulo.
* **vmed**: Velocidad media de los vehículos en el periodo de 15 minutos (km/h). Solo para puntos de medida interurbanos M30. Un valor negativo implica la ausencia de datos.
* **error**: Variable que indica si ha habido al menos una muestra errónea o sustituida en el laps de 15 minutos. Puede tomar los siguientes valores:
    * N: sin errores ni sustituciones.
    * E: la calidad de alguna de las muestras integradas no es óptima.
    * S: alguna de las muestras recibidas era totalmente errónea y no se ha integrado.
* **periodo_integracion**: Cantidad de muestras recibidas y consideradas para el periodo de integración.


In [115]:
# Enlace para descargar cada dataset con el histórico de datos del tráfico desde 2019
trafico_url = 'https://datos.madrid.es/egob/catalogo/208627-{id}-transporte-ptomedida-historico.zip'

# Empezamos cargando los datos de 2019-01
response = requests.get(trafico_url.format(id=68))

# En este caso, la descarga es un fichero zip
with zipfile.ZipFile(BytesIO(response.content), 'r') as zip_ref:
    # Dado que solo hay un archivo csv en el zip, seleccionamos el primero
    csv_file_name = zip_ref.namelist()[0]

    # Leemos el archivo csv que hay dentro del zip
    with zip_ref.open(csv_file_name) as csv_file:
        # Cargamos el archivo csv directamente con pandas
        trafico_raw_df = pd.read_csv(csv_file, delimiter=';', encoding='latin-1')

In [116]:
# Se comprueba la información básica del conjunto de datos cargado
trafico_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11203627 entries, 0 to 11203626
Data columns (total 9 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id                   int64  
 1   fecha                object 
 2   tipo_elem            object 
 3   intensidad           int64  
 4   ocupacion            float64
 5   carga                int64  
 6   vmed                 float64
 7   error                object 
 8   periodo_integracion  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 769.3+ MB


In [117]:
# Se revisan las primeras filas del conjunto de datos
trafico_raw_df.head()

,id,fecha,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,1001,2019-01-01 00:00:00,M30,2340,11.0,0,63.0,N,5
1,1001,2019-01-01 00:15:00,M30,2340,11.0,0,63.0,N,5
2,1001,2019-01-01 00:30:00,M30,2340,11.0,0,63.0,N,5
3,1001,2019-01-01 00:45:00,M30,2340,11.0,0,63.0,N,5
4,1001,2019-01-01 01:00:00,M30,2340,11.0,0,63.0,N,5


In [132]:
# Dado que los ficheros de histórico de tráfico son muy pesados, de más de 500 MB, para añadir los datos históricos de tráfico
# a cada accidente, se va a optar por hacer dos bucles anidados. En el primero de ellos, mes a mes se descarga el histórico de tráfico y carga
# los puntos de medida y los accidentes del mes en cuestión. El siguiente bucle es para bajar a nivel fecha_hora dentro de un mismo mes, de tal forma
# que se identifique para cada accidente, el punto de medida de tráfico más próximo en funcionamiento, y se añada dicha información
# al dataframe de accidentes

# Como paso inicial, se define la lista de fechas y el id del fichero csv para el primer bucle
fecha_inicio = datetime(2019, 1, 1)
fecha_fin = datetime(2023, 6, 1)
lista_AnoMes = [dt.strftime('%Y-%m') for dt in (fecha_inicio + timedelta(days=i) for i in range((fecha_fin - fecha_inicio).days+1)) if dt.day == 1]
lista_id_csv = range(68,122) # Desde enero de 2019 a junio de 2023

In [133]:
# A continuación, se define una función para invertir un diccionario, ya que se necesita invertir el diccionario utilizado para cargar los puntos de medida mes a mes
def invertir_diccionario(diccionario):
    diccionario_invertido = {valor: clave for clave, valor in diccionario.items()}
    return diccionario_invertido

# Se define el diccionario invertido de carga de los puntos de medida
# Esto será de necesario para cargar solo los puntos de medida del mes en cuestión según el primer bucle
dicc_pmed = invertir_diccionario(diccionario_pmed)

In [193]:
# El siguiente paso consiste en definir una función que permita identificar el punto de medida más cercano a cada accidente para una fecha_hora determinada
# Esta función se utilizará en el segundo bucle, ya que hay que hacer una búsqueda por fecha porque los puntos de medida más próximos pueden variar en función de la fecha,
# ya sea porque se han movido o porque están inactivos
def encuentra_pmed_mas_cercano(row, df2):
    df_misma_fecha = df2[df2['fecha'] == row['fecha_hora']]
    if df_misma_fecha.empty:
        return None
    dist_matrix = distance_matrix(row[['coordenada_x_utm', 'coordenada_y_utm']].values.reshape(1, -1), df_misma_fecha[['utm_x', 'utm_y']])
    id_pmed_mas_cercano = dist_matrix.argmin()

    return df_misma_fecha.iloc[id_pmed_mas_cercano]['id']

In [194]:
# El último paso previo al bucle es definir una lista vacía donde se cargarán los accidentes una vez añadida la información relativa al histórico de tráfico
accidentes_trafico_list = []

# Por último, se procede con el primer bucle que tiene en cuenta tanto el id del csv de los históricos de tráfico como el AnoMes correspondiente
for id, anomes in zip(lista_id_csv[48:], lista_AnoMes[48:]):
    response = requests.get(trafico_url.format(id=id))
    # En este caso, al igual que en la prueba anterior, la descarga es un fichero zip
    with zipfile.ZipFile(BytesIO(response.content), 'r') as zip_ref:
        # Dado que solo hay un archivo csv en el zip, seleccionamos el primero
        csv_file_name = zip_ref.namelist()[0]

        # Leemos el archivo csv que hay dentro del zip
        with zip_ref.open(csv_file_name) as csv_file:
            # Cargamos el archivo csv que contiene el histórico de tráfico del mes AnoMes en cuestión directamente con pandas
            df_temp_traf = pd.read_csv(csv_file, delimiter=';', encoding='latin-1')
            # Se obtienen los puntos de medida del AnoMes en cuestion
            for encoding in encodings:
                try:
                    df_temp_pmed = pd.read_csv(pmed_url.format(id=dicc_pmed[anomes]), delimiter=';', encoding=encoding, decimal='.')
                    break
                except UnicodeDecodeError:
                    continue
            # Hacemos un merge entre el histórico de tráfico y el de puntos de medida para georreferencial los registros de tráfico
            df_temp = df_temp_traf.merge(df_temp_pmed, left_on=['id'], right_on=['id'], how='left')
            # Se eliminan aquellos registros de df_temp que no tienen coordenadas
            df_temp.dropna(subset=['utm_x','utm_y'], inplace=True)
            df_temp['utm_x'] = df_temp['utm_x'].astype(float)
            df_temp['utm_y'] = df_temp['utm_y'].astype(float)
            # Se obtiene el dataframe de accidentes filtrando por el AnoMes en cuestion
            df_temp_accidentes_anomes = accidentes_raw_df[accidentes_raw_df['AnoMes']==anomes]
            # Se procede con el segundo bucle. En este caso, para cada AnoMes se hace un bucle sobre las distintas fecha_hora de los accidentes 
            # para así ver cuál es el registro de tráfico más próximo a cada accidente
            for f in df_temp_accidentes_anomes['fecha_hora'].unique():
                # Se crea un dataframe con los accidentes de la fecha_hora en cuestión
                df_temp_accidentes = df_temp_accidentes_anomes[df_temp_accidentes_anomes["fecha_hora"]==f].copy(deep=True)
                # Se cambian de formato las columnas de coordenadas
                df_temp_accidentes['coordenada_x_utm'] = df_temp_accidentes['coordenada_x_utm'].astype(float)
                df_temp_accidentes['coordenada_y_utm'] = df_temp_accidentes['coordenada_y_utm'].astype(float)
                # Se aplica la función definida anteriormente para obtener el punto de medida más cercado a cada accidente
                df_temp_accidentes['id_pmed_mas_cercano'] = df_temp_accidentes.apply(encuentra_pmed_mas_cercano, axis=1, df2=df_temp)
                # Se filtran aquellos accidentes que no tienen un punto de medida cercano activo
                df_temp_accidentes_clean = df_temp_accidentes.dropna(subset=['id_pmed_mas_cercano'])
                # Se muestra por pantalla si se han eliminado registros porque no hay ningún punto de medida cercano activo a los accidentes
                if (df_temp_accidentes.shape[0] - df_temp_accidentes_clean.shape[0]) != 0:
                    print(f"Se han eliminado {df_temp_accidentes.shape[0] - df_temp_accidentes_clean.shape[0]} registros porque no tienen punto de medida cercano para el año-mes {anomes} y fecha {f}")
                # Para los accidentes que sí tienen un punto de medida cercano activo...
                if not df_temp_accidentes_clean.empty:
                    # ... se añada la información relativa al histórico de accidentes
                    accidentes_pmed_df = df_temp_accidentes_clean.merge(df_temp, left_on=['fecha_hora', 'id_pmed_mas_cercano'], right_on=['fecha', 'id'], how='left')
                    # Se añade el registro resultante de añadir a cada accidente el histórico de tráfico a la lista
                    accidentes_trafico_list.append(accidentes_pmed_df)
                    # Se muestra en pantalla la fecha que se está analizando para hacer un seguimiento de la ejecución del segundo bucle
                    print("Seguimiento fecha: ", f, anomes)
            # Se concatenan los elementos de la lista para obtener un único dataframe resultante
            accidentes_trafico_df = pd.concat(accidentes_trafico_list)
            # Se guarda el dataframe resultante en un csv a modo de copia de seguridad, ya que solo nos interesa conservar el último fichero guardado
            accidentes_trafico_df.to_csv('./Datasets/accidentes_definitivo_hasta_'+anomes+'.csv', sep=';', encoding='latin-1', index=False)
            # Se muestra en pantalla el AnoMes actual del primer bucle
            print("Seguimiento anomes: ", anomes, csv_file_name)

Seguimiento fecha:  2023-01-02 07:45:00 2023-01
Seguimiento fecha:  2023-01-03 13:15:00 2023-01
Seguimiento fecha:  2023-01-03 15:45:00 2023-01
Seguimiento fecha:  2023-01-03 22:15:00 2023-01
Seguimiento fecha:  2023-01-04 11:00:00 2023-01
Seguimiento fecha:  2023-01-04 11:30:00 2023-01
Seguimiento fecha:  2023-01-04 20:30:00 2023-01
Seguimiento fecha:  2023-01-05 17:15:00 2023-01
Seguimiento fecha:  2023-01-06 11:00:00 2023-01
Seguimiento fecha:  2023-01-06 11:15:00 2023-01
Seguimiento fecha:  2023-01-08 20:00:00 2023-01
Seguimiento fecha:  2023-01-08 20:15:00 2023-01
Seguimiento fecha:  2023-01-09 12:45:00 2023-01
Seguimiento fecha:  2023-01-09 13:15:00 2023-01
Seguimiento fecha:  2023-01-09 15:15:00 2023-01
Seguimiento fecha:  2023-01-10 09:45:00 2023-01
Seguimiento fecha:  2023-01-12 07:00:00 2023-01
Seguimiento fecha:  2023-01-15 10:15:00 2023-01
Seguimiento fecha:  2023-01-15 13:15:00 2023-01
Seguimiento fecha:  2023-01-16 21:00:00 2023-01
Seguimiento fecha:  2023-01-19 18:30:00 

In [211]:
accidentes_trafico_df = pd.concat(backup+accidentes_trafico_list)

In [199]:
accidentes_trafico_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3921 entries, 0 to 0
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num_expediente           3921 non-null   object 
 1   fecha_x                  3921 non-null   object 
 2   hora                     3921 non-null   object 
 3   localizacion             3921 non-null   object 
 4   numero                   3921 non-null   object 
 5   cod_distrito             3921 non-null   int64  
 6   distrito_x               3921 non-null   object 
 7   tipo_accidente           3921 non-null   object 
 8   estado_meteorologico     3796 non-null   object 
 9   tipo_vehiculo            3921 non-null   object 
 10  tipo_persona             3921 non-null   object 
 11  rango_edad               3921 non-null   object 
 12  sexo                     3921 non-null   object 
 13  cod_lesividad            3399 non-null   float64
 14  lesividad                33

In [ ]:
# Se concatenan los elementos de la lista para obtener un único dataframe resultante
accidentes_trafico_df = pd.concat(accidentes_trafico_list)

In [ ]:
#accidentes_trafico_df = pd.read_csv('./Datasets/accidentes_definitivo_2019_2023.csv', delimiter=';', encoding='latin-1')

In [222]:
# Se analizan las columnas del dataframe definitivo, en aras de revisar cuáles hay que eliminar
accidentes_trafico_df.columns

Index(['num_expediente', 'fecha_x', 'hora', 'localizacion', 'numero',
       'cod_distrito', 'distrito_x', 'tipo_accidente', 'estado_meteorologico',
       'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
       'positiva_droga', 'AnoMes', 'fecha_hora', 'localizacion_1',
       'localizacion_2', 'clase_via_1', 'nombre_via_1', 'numero_via_1',
       'clase_via_2', 'nombre_via_2', 'numero_via_2', 'id_pmed_mas_cercano',
       'id', 'fecha_y', 'tipo_elem_x', 'intensidad', 'ocupacion', 'carga',
       'vmed', 'error', 'periodo_integracion', 'tipo_elem_y', 'distrito_y',
       'cod_cent', 'nombre', 'utm_x', 'utm_y', 'longitud', 'latitud',
       'implicados', 'implicados_peatones', 'implicados_pesados',
       'implicados_turismos', 'implicados_motocicletas',
       'implicados_bicicletas', 'implicados_otros', 'implicados_epac',
       'implicados_ligeros'],
      dtype='object')

In [225]:
# Se eliminan las columnas innecesarias
columnas_seleccionadas = ['num_expediente', 'fecha_x', 'hora', 'localizacion', 'numero',
       'localizacion_1', 'clase_via_1', 'nombre_via_1', 'numero_via_1',
       'localizacion_2', 'clase_via_2', 'nombre_via_2', 'numero_via_2',
       'cod_distrito', 'distrito_x', 'tipo_accidente', 'estado_meteorologico',
       'tipo_vehiculo', 'implicados', 'implicados_peatones', 'implicados_pesados', 'implicados_turismos',
       'implicados_motocicletas', 'implicados_bicicletas', 'implicados_otros',
       'implicados_epac', 'implicados_ligeros', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol', 'positiva_droga',
       'fecha_hora', 'id_pmed_mas_cercano', 'tipo_elem_x', 'intensidad', 'ocupacion', 'carga', 'vmed',
       'error', 'periodo_integracion', 'cod_cent', 'nombre', 'utm_x', 'utm_y']

accidentes_trafico_df = accidentes_trafico_df[columnas_seleccionadas]

In [226]:
# Se renombran las columnas
columnas_renombradas = ['num_expediente', 'fecha_acc', 'hora_acc', 'localizacion_acc', 'numero_acc',
       'localizacion_acc_1', 'clase_via_acc_1', 'nombre_via_acc_1', 'numero_via_acc_1',
       'localizacion_acc_2', 'clase_via_acc_2', 'nombre_via_acc_2', 'numero_via_acc_2',
       'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorologico',
       'tipo_vehiculo','implicados', 'implicados_peatones', 'implicados_pesados', 'implicados_turismos',
       'implicados_motocicletas', 'implicados_bicicletas', 'implicados_otros',
       'implicados_epac', 'implicados_ligeros', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
       'lesividad', 'utm_x_acc', 'utm_y_acc', 'positiva_alcohol', 'positiva_droga',
       'fecha_hora_med', 'id_pmed_mas_cercano', 'tipo_elem_pmed', 'intensidad',
       'ocupacion', 'carga', 'vmed', 'error', 'periodo_integracion',
       'cod_cent_pmed', 'nombre_pmed', 'utm_x_pmed', 'utm_y_pmed']
accidentes_trafico_df.columns = columnas_renombradas

In [232]:
# Se guarda el dataframe resultante en un csv
accidentes_trafico_df.to_csv('./Datasets/accidentes_definitivo_2019_2023_v3.csv', sep=';', encoding='latin-1', index=False)

In [233]:
# Observamos el tipo de dato de cada columna, así como el número de nulos
accidentes_trafico_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3921 entries, 0 to 3920
Data columns (total 49 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   num_expediente           3921 non-null   object 
 1   fecha_acc                3921 non-null   object 
 2   hora_acc                 3921 non-null   object 
 3   localizacion_acc         3921 non-null   object 
 4   numero_acc               3921 non-null   object 
 5   localizacion_acc_1       3921 non-null   object 
 6   clase_via_acc_1          3847 non-null   object 
 7   nombre_via_acc_1         3921 non-null   object 
 8   numero_via_acc_1         2043 non-null   object 
 9   localizacion_acc_2       3921 non-null   object 
 10  clase_via_acc_2          1612 non-null   object 
 11  nombre_via_acc_2         1707 non-null   object 
 12  numero_via_acc_2         15 non-null     object 
 13  cod_distrito             3921 non-null   int64  
 14  distrito                

In [234]:
# Se previsualiza el dataframe resultante
accidentes_trafico_df.head()

,num_expediente,fecha_acc,hora_acc,localizacion_acc,numero_acc,localizacion_acc_1,clase_via_acc_1,nombre_via_acc_1,numero_via_acc_1,localizacion_acc_2,...,intensidad,ocupacion,carga,vmed,error,periodo_integracion,cod_cent_pmed,nombre_pmed,utm_x_pmed,utm_y_pmed
0,2019S000036,02/01/2019,20:45:00,AVDA GRAN VIA DE HORTALEZA / GTA LUIS ROSALES,65B,AVDA GRAN VIA DE HORTALEZA,AVENIDA,GRAN VIA DE HORTALEZA,None,GTA LUIS ROSALES,...,373,8.0,16,0.0,N,15,56002,C/. López de Hoyos - Guisona-Gran Vía de Horta...,444606.203129,4.479884e+06
1,2019S000045,03/01/2019,10:30:00,"CTRA DEHESA DE LA VILLA, 1",1,"CTRA DEHESA DE LA VILLA, 1",CARRETERA,DEHESA DE LA VILLA1,None,,...,60,6.0,7,0.0,N,14,29012,(TACTICO) AV. COMPLUTENSE S-N (GIRO A DEHESA D...,438132.271587,4.478716e+06
2,2019S000132,03/01/2019,12:45:00,AVDA SANTA EUGENIA / CALL REAL DE ARGANDA,64,AVDA SANTA EUGENIA,AVENIDA,SANTA EUGENIA,None,CALL REAL DE ARGANDA,...,415,8.0,48,0.0,N,15,88006,C/. Peñaranda Bracamonte - Av. Entrepeñas-Av. ...,448457.948911,4.469188e+06
3,2019S000132,03/01/2019,12:45:00,AVDA SANTA EUGENIA / CALL REAL DE ARGANDA,64,AVDA SANTA EUGENIA,AVENIDA,SANTA EUGENIA,None,CALL REAL DE ARGANDA,...,415,8.0,48,0.0,N,15,88006,C/. Peñaranda Bracamonte - Av. Entrepeñas-Av. ...,448457.948911,4.469188e+06
4,2019S000133,03/01/2019,14:30:00,"CALL FELIPE ALVAREZ, 10",10,"CALL FELIPE ALVAREZ, 10",CALLE,FELIPE ALVAREZ,10,,...,428,52.0,62,0.0,N,15,82027,C/. Felipe Alvarez - Manuel Pavia-Jesus del Pino,447076.418439,4.470348e+06


In [235]:
# Se analiza cuántos accidentes son urbanos y cuántos interurbanos
accidentes_trafico_df.tipo_elem_pmed.value_counts()

URB    3593
M30     311
C30      17
Name: tipo_elem_pmed, dtype: int64

In [236]:
# Se calcula qué porcentaje de los accidentes no tiene información relativa al tráfico
print(f"Un {((accidentes_raw_df.count()[0] - accidentes_trafico_df.count()[0]) / accidentes_raw_df.count()[0])*100:.4f}% de los registros no tiene información de tráfico")

Un 2.4141% de los registros no tiene información de tráfico
